# Course 1: Lidar
## Part 3: Clustering Obstacles
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Sensor Fusion Nanodegree programme offered at Udacity.

## Objectives

## 1. Introduction

### 1.1: Euclidean Clustering with KD-Tree

#### Euclidean clustering algorithm

Euclidean clustering is a type of [cluster analysis](https://en.wikipedia.org/wiki/Cluster_analysis) algorithm used to detect and group nearby points in a dataset. Using a [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) heuristic, each point's membership to its neighbours is considered. Points which are "close" in distance to each otoher are grouped together to form a local cluster. Through the use of a [nearest neighbour search](https://en.wikipedia.org/wiki/Nearest_neighbor_search), this "closeness" heuristic is explored, and the resulting set of clusters is approximated in the fashion of an optimisation problem. In other words, the "most optimal" set of resulting clusters is determined by grouping together the points whose distance between each other is minimal.

In our application, we use Euclidean clustering to group the LiDAR points belonging to the distinct objects in our scene — the surrounding vehicles. We make use of the Point Cloud Library (PCL), which provides us with the [`pcl::EuclideanClusterExtraction`](https://pointclouds.org/documentation/tutorials/cluster_extraction.html) class to perform the clustering. Using a [K-D Tree](https://en.wikipedia.org/wiki/K-d_tree) allows us to "speed up" the nearest neighbour search by organising points into a hierarchial strucutre, therefore restricting the number of total points that must be "explored" when forming each cluster. The K-D Tree is a special case of the [binary space partitioning](https://en.wikipedia.org/wiki/Binary_space_partitioning) tree structure, which "splits" the point space into branches based on the median values of the points along the chosen dimension(s). Once the tree is formed using the entire set of possible points, "searching" for nearby points becomes a matter of tree traversal, where the distance is checked against the nodes of the tree in a recursive manner. With the K-D Tree structure, we are able to more-efficiently perform "look-up" in $O\left(\mathrm{log}n\right)$ time complexity.

## 2. Programming Task

### Euclidean Clustering with KD-Tree using Point Cloud Library (PCL)

#### E1.3.1: `Clustering`

By following the [Euclidean Cluster Extraction](https://pointclouds.org/documentation/tutorials/cluster_extraction.html) tutorial from Point Cloud Library (PCL), we will be able to group together individual LiDAR point returns to form "clusters". These clusters can be then used to identify objects in the driving scene, such as other vehicles, pedestrians or VRUs (vulnerable road users, e.g., bicyclists).  

##### The `Clustering` function

```cpp
// From J. Moran's `src/processPointClouds.cpp`
// Credit: https://github.com/jonathanloganmoran/ND313-Sensor-Fusion-Engineer/blob/1.3/1-Lidar/1-1-Lidar-Obstacle-Detection/src/processPointClouds.cpp
```

```cpp
// In `src/processPointClouds.cpp`:


template<typename PointT> std::vector<
    typename pcl::PointCloud<PointT>::Ptr
> ProcessPointClouds<PointT>::Clustering(
    typename pcl::PointCloud<PointT>::Ptr cloud,
    float clusterTolerance,
    int minSize,
    int maxSize
) {
    // Time clustering process
    auto startTime = std::chrono::steady_clock::now();
    std::vector<typename pcl::PointCloud<PointT>::Ptr> clusters;
    /*** E1.3.1: Euclidean clustering with PCL. ***/
    // TODO:: Fill in the function to perform euclidean clustering to group detected obstacles
    // Creating the KD-Tree object for the search method of the extraction
    pcl::search::KdTree<pcl::PointXYZ>::Ptr tree(
        new pcl::search::KdTree<pcl::PointXYZ>
    );
    // Creating the Euclidean clustering class instance
    pcl::EuclideanClusterExtraction<pcl::PointXYZ> ec;
    // Setting the input cloud for the KD-Tree
    // NOTE: We assume the ground plane has been "filtered" out
    tree->setInputCloud(cloud);
    // Configuring the clustering parameters
    std::vector<pcl::PointIndices> clusterIndices;
    ec.setClusterTolerance(clusterTolerance);
    ec.setMinClusterSize(minSize);
    ec.setMaxClusterSize(maxSize);
    ec.setSearchMethod(tree);
    ec.setInputCloud(cloud);
    ec.extract(clusterIndices);
    // Performing the clustering with Euclidean distance
    for (const auto& cluster : clusterIndices) {
        // Creating a new point cloud instance for the current cluster
        pcl::PointCloud<pcl::PointXYZ>::Ptr cloudCluster(
            new pcl::PointCloud<pcl::PointXYZ>
        );
        for (const auto& idx : cluster.indices) {
            // Copying over the indices of the current cluster
            cloudCluster->push_back(
                (*cloud)[idx]
            );
        }
        // Setting the cluster parameters
        cloudCluster->width = cloudCluster->size();
        cloudCluster->height = 1;
        cloudCluster->is_dense = true;
        std::cout << "PointCloud representing the Cluster: "
                  << cloudCluster->size() << " data points.\n";
        // Adding cluster to return vector
        clusters.push_back(cloudCluster);
    }
    auto endTime = std::chrono::steady_clock::now();
    auto elapsedTime = std::chrono::duration_cast<
        std::chrono::milliseconds
    >(endTime - startTime);
    std::cout << "Clustering took " 
              << elapsedTime.count() << " milliseconds,"
              << " and found " << clusters.size() << " clusters.\n";
    return clusters;
}
```

##### Testing the `Clustering` function

To run the above `Clustering` algorithm, call the function from inside `src/environment.cpp` as follows:

```cpp
// From J. Moran's `src/environment.cpp`:
// Credit: https://github.com/jonathanloganmoran/ND313-Sensor-Fusion-Engineer/blob/1.3/1-Lidar/1-1-Lidar-Obstacle-Detection/src/environment.cpp
```

```cpp
// In `src/environment.cpp`:

void simpleHighway(
    pcl::visualization::PCLVisualizer::Ptr& viewer
) {
    // ..
    /** E1.3.1: Euclidean clustering with PCL. **/
    // Defining the clustering parameters
    double distanceTol = 1.0;           // Euclidean threshold (m)
    int minSize = 3;
    int maxSize = 30;
    // Performing the clustering
    std::vector<
        pcl::PointCloud<pcl::PointXYZ>::Ptr
    > cloudClusters = pointProcessorXYZ.Clustering(
        segmentCloud.first,
        distanceTol,
        minSize,
        maxSize
    );
    // ..
}
```

We define our input arguments to the `Clustering` function:
* `distanceTol` — Threshold distance (in metres) for determining point candidates, i.e., whether points should be considered part of the same cluster. Points separated by a greater distance than this threshold will not be grouped together;
* `minSize` — The minimum number of points assumed to belong to an individual cluster. By setting a "minimum" number of points for each "cluster", we can prevent forming clusters which are too sparse, which might be related to noise and not representative of actual objects in the scene;
* `maxSize` — The maximum number of points assumed to belong to an individual cluster. By setting a "maximum" number of points for each "cluster", we can prevent forming clusters which are too dense, which might erroneously combine points from multiple distinct objects.

By carefully selecting these parameters, we can attempt to avoid clustering points which might have been generated from noise (either from sensor inaccuracies) or are belonging to non-objects or multiple overlapping objects.

#### E1.3.2: Visualising the clusters

In this part we simply render the clusters we generated above using the `Clustering` function. To display these point clusters, we will use the PCL Viewer canvas. To make the clusters "stand out", we will assign each a unique RGB-valued colour, then render the points onto the canvas.

```cpp
// From J. Moran's `src/environment.cpp`:
// Credit: https://github.com/jonathanloganmoran/ND313-Sensor-Fusion-Engineer/blob/1.3/1-Lidar/1-1-Lidar-Obstacle-Detection/src/environment.cpp
```

```cpp
// In `src/environment.cpp`:

void simpleHighway(
    pcl::visualization::PCLVisualizer::Ptr& viewer
) {
    // ..
    /** E1.3.2: Visualising the clusters found. **/
    // Using a unique identifer for each cluster found
    int clusterId = 0;
    // Set of colours to use when rendering the clusters found
    std::vector<Color> colors = {
        Color(1, 0, 0),
        Color(0, 1, 0),
        Color(0, 0, 1)
    };
    // Rendering each cluster onto the PCL viewer
    for (pcl::PointCloud<pcl::PointXYZ>::Ptr cluster : cloudClusters) {
        std::cout << "cluster size: ";
        pointProcessorXYZ.numPoints(cluster);
        renderPointCloud(
            viewer,
            cluster,
            "obstCloud" + std::to_string(clusterId),
            colors[clusterId]
        );
        ++clusterId;
    }
    // ..
}
```

## 3. Closing Remarks

#### Alternatives
*


#### Extensions of task
*

## 4. Future Work
* ⬜️
* ✅

## Credits

This assignment was prepared by Aaron Brown and Michael Maile of Mercedes-Benz Research & Development of North America (MBRDNA), 2021 (link [here](https://learn.udacity.com/nanodegrees/nd313/)).


References
* [1] .


Helpful resources:
* []().